In [ ]:
%pip install --upgrade --force-reinstall transformers==4.52.4

In [ ]:
%pip install -q transformers datasets scikit-learn pandas accelerate

In [ ]:
from transformers import TrainingArguments

In [ ]:
args = TrainingArguments(output_dir="./results")
# print(args)


In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
!pip install openpyxl


In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
df=pd.read_excel("1.xlsx")

In [ ]:
# Merge question + text
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
df['labels'] = df['categoriesString'].apply(lambda x: [c.strip() for c in x.split(',')])

In [ ]:
# Drop rows where the 'text' column is NaN
df = df.dropna(subset=["questionText", "answer", "labels"])

# Optionally, reset the index after dropping rows (if needed)
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df=df[["questionText", "answer","labels"]]

In [ ]:
# Initialize the encoder
mlb = MultiLabelBinarizer()

# Fit and transform the categories to boolean values
encoded = mlb.fit_transform(df['labels'])

# Create a new DataFrame with the encoded categories
encoded_df = pd.DataFrame(encoded, columns=mlb.classes_, index=df.index)

# Concatenate with the original DataFrame (excluding 'categories' column)
df_encoded = pd.concat([df.drop('labels', axis=1), encoded_df], axis=1)

In [ ]:
df=df_encoded

In [ ]:
y = df.drop(columns=["questionText", "answer"])

In [ ]:
minority_columns = y.sum()

In [ ]:
top10 = minority_columns.nlargest(1).index.tolist()

In [ ]:
top10

In [ ]:
minority_columns[["Entlohnung"]]

In [ ]:
# 10 posto
below_94 = minority_columns[minority_columns < 94].index.tolist()

print(below_94)


In [ ]:
from sklearn.utils import resample
import pandas as pd

def collect_and_pad_positive_rows(df, columns, target_n=94):
    """
    Collects all rows where each column == 1, and upscales to target_n rows per column.

    Args:
        df (pd.DataFrame): The full dataset
        columns (list): List of binary column names to check (e.g., ["x", "y"])
        target_n (int): Minimum number of rows to keep per class

    Returns:
        dict: Dictionary with keys = column names and values = padded DataFrames
    """
    results = {}

    for col in columns:
        subset = df[df[col] == 1].copy()
        n_rows = len(subset)

        if n_rows < target_n:
            # Upsample with replacement
            subset = resample(
                subset,
                replace=True,
                n_samples=target_n,
                random_state=42
            )

        results[col] = subset

    return results


In [ ]:
columns_to_check = below_94
my_result = collect_and_pad_positive_rows(df, columns_to_check, target_n=94)


In [ ]:
for col, df_upsampled in my_result.items():
    count = df_upsampled[col].sum()
    total = len(df_upsampled)
    print(f"{col}: total rows = {total}, positive labels = {count}")


In [ ]:
# Combine all upsampled DataFrames from your dict
padded_df = pd.concat(my_result.values()).reset_index(drop=True)


In [ ]:
# 1. Remove original minority rows
df_filtered = df.copy()
for col in columns_to_check:
    df_filtered = df_filtered[df_filtered[col] != 1]

# 2. Combine the filtered df with the upsampled data
df = pd.concat([df_filtered, padded_df]).reset_index(drop=True)


In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Features and labels
y = df.drop(columns=["questionText", "answer"])
X = df[["questionText", "answer"]]

# First split: separate out the test set (10%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# Second split: from remaining 90%, take 10% for validation (so 9% total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1111, random_state=42
)

In [ ]:
# 4. Tokenization
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_texts(text_a, text_b, labels):
    encodings = tokenizer(
        text=text_a.tolist(),
        text_pair=text_b.tolist(),
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    encodings['labels'] = torch.tensor(labels, dtype=torch.float)
    return encodings

# Train encodings
train_encodings = tokenize_texts(
    X_train["questionText"],
    X_train["answer"],
    y_train.values
)

# Validation encodings (NEW)
val_encodings = tokenize_texts(
    X_val["questionText"],
    X_val["answer"],
    y_val.values
)

# Test encodings
test_encodings = tokenize_texts(
    X_test["questionText"],
    X_test["answer"],
    y_test.values
)


In [ ]:
class SurveyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = SurveyDataset(train_encodings)
val_dataset = SurveyDataset(val_encodings)
test_dataset = SurveyDataset(test_encodings)

In [ ]:
#6. Model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=y.shape[1],
    problem_type="multi_label_classification"
)

In [ ]:
# 🔒 Freeze lower layers and unfreeze top 4 + classifier
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if any(layer in name for layer in [
        "encoder.layer.8",
        "encoder.layer.9",
        "encoder.layer.10",
        "encoder.layer.11"
    ]):
        param.requires_grad = True

    if "classifier" in name:
        param.requires_grad = True


In [ ]:
from transformers import Trainer
import torch.nn as nn
#Custom trainer u kojme se loss funkcija kustomizita ia ubacuje wieght
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Threshold grid
thresholds_to_try = np.arange(0.1, 0.9, 0.01)

def compute_metrics(pred):
    logits, true_labels = pred

    # Ensure numpy arrays (avoid creating a torch.Tensor each time)
    if not isinstance(logits, np.ndarray):
        try:
            logits = np.asarray(logits)
        except Exception:
            logits = np.array(logits)  # fallback

    if not isinstance(true_labels, np.ndarray):
        true_labels = np.asarray(true_labels)

    # Sigmoid to get probabilities (vectorized)
    probs = 1 / (1 + np.exp(-logits))  # equivalent to torch.sigmoid

    num_labels = true_labels.shape[1]
    best_thresholds = np.zeros(num_labels, dtype=float)
    final_preds = np.zeros_like(true_labels)

    # Per-label threshold search
    for i in range(num_labels):
        # Broadcast comparisons and metric calculation
        best_f1 = -1.0
        best_thresh = 0.5
        for thresh in thresholds_to_try:
            preds_i = (probs[:, i] > thresh).astype(int)
            f1 = f1_score(true_labels[:, i], preds_i, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh
        best_thresholds[i] = best_thresh
        final_preds[:, i] = (probs[:, i] > best_thresh).astype(int)

    # Compute aggregated metrics
    f1_micro = f1_score(true_labels, final_preds, average="micro", zero_division=0)
    f1_macro = f1_score(true_labels, final_preds, average="macro", zero_division=0)
    precision_micro = precision_score(true_labels, final_preds, average="micro", zero_division=0)
    recall_micro = recall_score(true_labels, final_preds, average="micro", zero_division=0)

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
        # Convert to plain Python list so Trainer can serialize without error
        "best_thresholds": best_thresholds.tolist(),
    }


In [ ]:
# 7. Training
from transformers import TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./output_model",

    # ✅ Corrected parameter name
    eval_strategy="epoch",    # evaluates at the end of each epoch
    save_strategy="epoch",          # saves model checkpoint each epoch

    # ✅ Learning parameters (still flexible to tune later)
    learning_rate=2e-5,             # can be tuned later or scheduled dynamically
    lr_scheduler_type="linear",     # gradually decreases LR over time
    warmup_ratio=0.1,               # 10% of steps for learning rate warmup

    # ✅ Batch & epoch settings
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=11,
    weight_decay=0.01,

    # ✅ Logging & saving
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,

    # ✅ Best model & metric
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    push_to_hub=False,        # prevents uploading anything to HF Hub
    report_to=[]
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,           # ✅ uses validation set (not test)
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

    # ✅ Early stopping to avoid overfitting
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
import os
import json
import shutil

# === Prepare directory ===
local_dir = "./my_trained_model"
os.makedirs(local_dir, exist_ok=True)

# === 1. Save the best model + tokenizer + training args ===
# (If training finished normally, trainer.model is already the best model)
trainer.save_model(local_dir)
tokenizer.save_pretrained(local_dir)

with open(os.path.join(local_dir, "training_args.json"), "w") as f:
    f.write(trainer.args.to_json_string())

# === 2. Save label names ===
label_names = y.columns.tolist()  # y should be your label DataFrame
with open(os.path.join(local_dir, "label_names.json"), "w") as f:
    json.dump(label_names, f, indent=2)

# === 3. Find thresholds for the best model ===
threshold_best = None
metric_key = "eval_" + trainer.args.metric_for_best_model
best_metric = trainer.state.best_metric

for log in trainer.state.log_history:
    if metric_key in log and abs(log[metric_key] - best_metric) < 1e-12:
        if "eval_best_thresholds" in log:
            threshold_best = log["eval_best_thresholds"]
        break

if threshold_best is not None:
    with open(os.path.join(local_dir, "best_thresholds.json"), "w") as f:
        json.dump(threshold_best, f, indent=2)
    print("✅ Saved thresholds matching the best model.")
else:
    print("⚠️ No thresholds found for the best model.")

# === 4. Save log history (optional but useful) ===
with open(os.path.join(local_dir, "log_history.json"), "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)

# === 5. Zip and download (for Colab) ===
zip_path = shutil.make_archive("my_trained_model", "zip", local_dir)
print(f"📦 Zipped to: {zip_path}")

try:
    from google.colab import files
    files.download(zip_path)
except Exception:
    print("✅ Saved locally (not in Colab).")


In [ ]:
import json

# Load log history from file
with open("./my_trained_model/log_history.json", "r") as f:
    log_history = json.load(f)

# Print evaluation entries (with metrics like F1, precision, thresholds, etc.)
print("📊 Evaluation Metrics per Epoch:\n")
for entry in log_history:
    if "eval_loss" in entry:  # Only show evaluation steps
        print(f"Epoch {entry.get('epoch', '?')} (Step {entry.get('step', '?')}):")
        print(f"  - Eval Loss         : {entry.get('eval_loss'):.4f}")
        print(f"  - F1 Micro          : {entry.get('eval_f1_micro'):.4f}")
        print(f"  - F1 Macro          : {entry.get('eval_f1_macro'):.4f}")
        print(f"  - Precision (Micro) : {entry.get('eval_precision_micro'):.4f}")
        print(f"  - Recall (Micro)    : {entry.get('eval_recall_micro'):.4f}")
        print(f"  - Thresholds (len)  : {len(entry.get('eval_best_thresholds', []))} labels")
        print("-" * 50)


In [ ]:
import json

with open("./my_trained_model/best_thresholds.json", "r") as f:
    thresholds = json.load(f)

print("📊 Best thresholds from last epoch:")
for i, t in enumerate(thresholds):
    print(f"Label {i}: Threshold = {t}")

